In [19]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.net.usercache.abstract_usercache as enua
import emission.storage.timeseries.timequery as estt
import numpy as np

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a"), UUID("95e70727-a04e-3e33-b7fe-34ab19194f8b")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73"), UUID("3bc0f91f-7660-34a2-b005-5c399598a369")]

In [20]:
list(edb.get_timeseries_db().find({"metadata.key": "config/sensor_config", "data.battery_status": {"$exists": True}}).sort("metadata.write_ts").limit(10))

[{u'_id': ObjectId('56fadecd88f66341cb3618c9'),
  u'data': {u'android_health': u'GOOD',
   u'android_plugged': u'USB',
   u'android_technology': u'Li-ion',
   u'android_temperature': 226,
   u'android_voltage': 226,
   u'battery_level_pct': 100,
   u'battery_status': 5},
  u'metadata': {u'key': u'config/sensor_config',
   u'platform': u'android',
   u'read_ts': 1459281599.629,
   u'time_zone': u'America/Los_Angeles',
   u'type': u'sensor-data',
   u'write_fmt_time': u'2016-03-29T11:56:24.694000-07:00',
   u'write_local_dt': datetime.datetime(2016, 3, 29, 11, 56, 24, 694000),
   u'write_ts': 1459277784.694},
  u'user_id': UUID('e471711e-bd14-3dbe-80b6-9c7d92ecc296')},
 {u'_id': ObjectId('56faecf288f66323bcc68ee9'),
  u'data': {u'android_health': u'GOOD',
   u'android_plugged': u'USB',
   u'android_technology': u'Li-ion',
   u'android_temperature': 197,
   u'android_voltage': 197,
   u'battery_level_pct': 100,
   u'battery_status': 2},
  u'metadata': {u'key': u'config/sensor_config',
   

In [21]:
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

In [22]:
ranges_df = pd.read_csv("../results_spring_2016/ranges.csv")

In [23]:
ranges_df

,start_ts,start_fmt_time,end_ts,end_fmt_time,state,android_regime,ios_regime
0,1459302151,2016-03-29T18:42:30.685000-07:00,1459310790,2016-03-29T21:06:30.275000-07:00,moving,high+1sec,best+1m
1,1459310790,2016-03-29T21:06:30.275000-07:00,1459342810,2016-03-30T06:00:09.654643-07:00,stationary,high+1sec,best+1m
2,1459342810,2016-03-30T06:00:09.654643-07:00,1459353614,2016-03-30T09:00:13.859000-07:00,charging-or-full,high+1sec,best+1m
3,1459353614,2016-03-30T09:00:13.859000-07:00,1459371719,2016-03-30T14:01:58.965395-07:00,charging-or-full,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m
4,1459371719,2016-03-30T14:01:58.965395-07:00,1459372435,2016-03-30T14:13:55.019000-07:00,loitering,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m
5,1459372435,2016-03-30T14:13:55.019000-07:00,1459373082,2016-03-30T14:24:42.464000-07:00,moving,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m
6,1459373082,2016-03-30T14:24:42.464000-07:00,1459376488,2016-03-30T15:21:27.670000-07:00,loitering,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m
7,1459376488,2016-03-30T15:21:27.670000-07:00,1459377222,2016-03-30T15:33:41.654000-07:00,moving,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m
8,1459377222,2016-03-30T15:33:41.654000-07:00,1459379999,2016-03-30T16:19:58.943000-07:00,loitering,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m
9,1459379999,2016-03-30T16:19:58.943000-07:00,1459381062,2016-03-30T16:37:41.978000-07:00,moving,high-v-balanced+1sec-v-30sec,best-v-10m+1m-v-30m


In [24]:
regime_grouped_ranges_df = ranges_df.groupby(['state', 'android_regime', 'ios_regime'])

In [25]:
#invalid_df = pd.read_csv("../results_spring_2016/invalid.csv")

In [26]:
#invalid_df

In [27]:
def get_points_from_ts_for_range(ts, start_ts, end_ts):
    time_range = estt.TimeQuery("metadata.write_ts", start_ts, end_ts)
    return ts.get_data_df("background/location", time_range)

In [28]:
def get_points_for_range(ts, range_df):
    points_range_df = map(lambda row: get_points_from_ts_for_range(ts, row['start_ts'], row['end_ts']), range_df.to_dict('records'))
    return points_range_df

In [29]:
def get_points_for_group(ts_list, range_df):
    return map(lambda ts: get_points_for_range(ts, range_df), ts_list)

In [30]:
iphone_high_fast_stationary_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'stationary' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(iphone_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        iphone_high_fast_stationary_df = iphone_high_fast_stationary_df.append(concat_phone_location_df_list)

In [37]:
android_high_fast_stationary_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'stationary' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(android_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        android_high_fast_stationary_df = android_high_fast_stationary_df.append(concat_phone_location_df_list)
        #android_high_fast_stationary_df = android_high_fast_stationary_df.sample(frac=0.001)

In [38]:
import emission.analysis.plotting.leaflet_osm.our_plotter as lo
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy

In [39]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(iphone_high_fast_stationary_df)]) + 
                 lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(android_high_fast_stationary_df)])])

KeyError: 'dummy_entry'

In [ ]:
iphone_high_fast_loitering_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'loitering' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(iphone_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        iphone_high_fast_loitering_df = iphone_high_fast_loitering_df.append(concat_phone_location_df_list)

In [14]:
android_high_fast_loitering_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'loitering' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(android_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        android_high_fast_loitering_df = android_high_fast_loitering_df.append(concat_phone_location_df_list)
        android_high_fast_loitering_df = android_high_fast_loitering_df.sample(frac=0.01)

In [18]:
reload(gfc)

<module 'emission.analysis.plotting.geojson.geojson_feature_converter' from '/Users/shankari/e-mission/e-mission-server/emission/analysis/plotting/geojson/geojson_feature_converter.pyc'>

In [19]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(iphone_high_fast_loitering_df)]) + 
                 lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(android_high_fast_loitering_df)])])

DEBUG:root:Getting midpoint of (-122.2590450012332, 37.87540032531868) and (-122.2587817126735, 37.87534147657365)


Found 579 points
Found 391 points

DEBUG:root:Getting midpoint of (-122.2589475, 37.8751869) and (-122.2585713, 37.8754273)
